In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import nltk
import email
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
np.random.seed(49)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#df1 = pd.read_csv("/kaggle/input/emailsclean/New_emailsdata.csv")
df1 = pd.read_csv("/kaggle/input/enroncleanobff/EnronDatasetProjectNew.csv")
df2 = pd.read_csv("/kaggle/input/bbc-text-clean/BBC-Text-Clean.csv")

In [3]:
df1.head()

,Unnamed: 0,label,category,text,emailclean,taux_obfuscation
0,0,0,business,fyi \n forwarded by lauri...,fyi forwarded by lauri allen hou ect on pm kim...,0.000673
1,1,0,business,jackie \nsince the inlet to river plant is sh...,jackie since the inlet to river plant is shut ...,0.001221
2,2,0,politics,daren sorry i forgot to include you\n ...,daren sorry forgot to include you forwarded by...,0.001010
3,3,0,politics,pursuant to our recent confirmation i have dra...,pursuant to our recent confirmation have draft...,0.001546
4,4,0,sport,details for the deal to be entered on jan ...,details for the deal to be entered on jan buy ...,0.002096


In [4]:
unique_categories = df1["category"].unique()
print(unique_categories)


['business' 'politics' 'sport' 'entertainment' 'tech']


In [5]:
# Importer la bibliothèque Pandas
import pandas as pd

# Compter le nombre d'instances pour chaque catégorie
counts = df1["category"].value_counts()

# Afficher le nombre d'instances pour chaque catégorie
for category, count in counts.items():
    print("Catégorie :", category, " - Nombre d'instances :", count)

Catégorie : entertainment  - Nombre d'instances : 1549
Catégorie : politics  - Nombre d'instances : 1256
Catégorie : business  - Nombre d'instances : 1189
Catégorie : sport  - Nombre d'instances : 1115
Catégorie : tech  - Nombre d'instances : 1001


In [6]:
# Supprimer les instances avec emailclean vide ou NaN
df1 = df1.dropna(subset=['emailclean'])
df1 = df1[df1['emailclean'] != '']

# Réinitialiser les index
df1.reset_index(drop=True, inplace=True)

In [7]:
df1.shape

(6110, 6)

partie détéction et calcule du taux d'obfuscation

In [8]:
import email
import matplotlib.pyplot as plt
import re
def extract_body(email_message):
    if email_message.is_multipart():
        # If the email has multiple parts, extract the text from the first part
        body = email_message.get_payload(0).get_payload()
    else:
        body = email_message.get_payload()
    return body

emails = df3['texteTest'].tolist()
bodies = []

for email_str in emails:
    email_message = email.message_from_string(email_str)
    body = extract_body(email_message)
    bodies.append(body)

def detect_obfuscation(body):
    # Obfuscation detection patterns
    patterns = [
    r"\b(?:make money|earn money|have money|get rich)\b",  # Phrases liées à l'argent
    r"\b(?:lose weight|diet|fitness)\b",  # Phrases liées à la perte de poids ou au régime
    r"\b(?:free|discount|offer)\b",  # Phrases liées aux offres gratuites ou aux réductions
    r"\b(?:limited time|exclusive deal)\b",  # Phrases liées aux offres limitées dans le temps ou exclusives
    r"\b(?:lottery|prize|win)\b",  # Phrases liées aux loteries, prix ou gains
    r"\b(?:urgent|important|attention)\b",  # Phrases avec des termes d'urgence ou d'importance
    r"\b(?:viagra|cialis|medication)\b",  # Phrases liées à des médicaments ou produits pharmaceutiques
    r"\b(?:credit card|bank account)\b",  # Phrases liées aux cartes de crédit ou aux comptes bancaires
    r"\b(?:refund|money back)\b",  # Phrases liées aux remboursements ou aux retours d'argent
    r"\b(?:exclusive offer|secret formula)\b",  # Phrases liées à des offres exclusives ou à des formules secrètes
    r"\b[\w\.-]+@[\w\.-]+\b",  # Simple email address
    r"\b(?:https?://)?(?:[A-Za-z0-9-]+\.)*[A-Za-z0-9-]+\.[A-Za-z]{2,}\S*\b",  # URL
    r"\\[uU][a-fA-F0-9]{4}",  # Unicode characters
    "[^\w\s]",  # Modèles de caractères spéciaux
    r"\d",  # Modèles de nombres
    r"\s{2,}",  # Modèles d'espaces inattendus
    r"pharmacie en ligne|gratuit|promotion|gagner de l'argent rapidement",  # Modèles de mots-clés
    r"bit\.ly|goo\.gl",  # Modèles de liens raccourcis
    r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",  # Modèles d'adresses IP
    r"\b[a-zA-Z]+[^.?!]*[.?!]",  # Modèles d'erreurs grammaticales
    r"[^.?!]*[.?!]",  # Modèles de phrases incomplètes
    r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b",  # Modèles d'adresses e-mail
    r"\b\d{3}[-.]?\d{3}[-.]?\d{4}\b",  # Modèles de numéros de téléphone
    r"\b\d+\s+[A-Za-z]+\s+[A-Za-z]+\b",  # Modèles d'adresses postales
    ]
 

    # Search for obfuscation patterns
    obfuscated = []
    for pattern in patterns:
        obfuscated += re.findall(pattern, body)

    # Return True if obfuscation patterns were found, False otherwise
    return bool(obfuscated)

obfuscated_emails = []

for body in bodies:
    if detect_obfuscation(body):
        obfuscated_emails.append(body)

print(len(obfuscated_emails))

NameError: name 'df3' is not defined

In [ ]:
# calculer le taux d'obfuscation pour chaque email

taux_obfuscation = []

for email_str in df3['texteTest']:

    email_message = email.message_from_string(email_str)

    body = extract_body(email_message)

    nb_obfuscations = detect_obfuscation(body)

    taux_obfuscation.append(nb_obfuscations / len(body) if len(body) > 0 else 0)  # éviter la division par zéro



# ajouter la colonne "taux_obfuscation" au DataFrame

df3['taux_obfuscation'] = taux_obfuscation

In [ ]:
spam_count = df1[df1["label"] == 1].shape[0] 
ham_count = df1[df1["label"] == 0].shape[0]

print("Nombre d'instances spam :", spam_count) 
print("Nombre d'instances ham :", ham_count)

PARTIE CLASSIFICATION DE SPAM

In [ ]:
import tensorflow_hub as hub
# Chargement du modèle Universal Sentence Encoder
model_path = "/kaggle/input/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/2"
embed = hub.load(model_path)

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
# Transforming catagories label to 
labels = df1['category'].astype(str)

lb = LabelBinarizer().fit(list(set(labels.tolist())))
train_labels_categoryCat = lb.transform(df1['category'].tolist())

In [ ]:
train_texts_category = df1['emailclean']
train_labels_category = df1['label']
train_category_obf_rate = df1['taux_obfuscation']

    # Diviser les données en ensembles d'entraînement, de validation et de test
train_texts, test_texts, train_labels, test_labels, train_cat, test_cat, train_obf_rate, test_obf_rate = train_test_split(
train_texts_category, train_labels_category, train_labels_categoryCat, train_category_obf_rate,
test_size=0.3, stratify=train_labels_category, random_state=42)

train_texts, val_texts, train_cat, val_cat, train_labels, val_labels, train_obf_rate, val_obf_rate = train_test_split(
train_texts, train_cat, train_labels, train_obf_rate,
test_size=0.3, stratify=train_labels, random_state=42)

In [ ]:
# Préparer les données d'entraînement, de validation et de test avec l'embedding Universal Sentence Encoder
train_features = embed.signatures['serving_default'](tf.convert_to_tensor(train_texts))['outputs']
val_features = embed.signatures['serving_default'](tf.convert_to_tensor(val_texts))['outputs']
test_features = embed.signatures['serving_default'](tf.convert_to_tensor(test_texts))['outputs']


In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Attention
# Définir les couches d'entrée pour les textes et le taux d'obfuscation
input_text = tf.keras.layers.Input(shape=(train_features.shape[1],), name='text')#sa permet au model de s'adapter a la dimensionalité d'origine
input_obf_rate = tf.keras.layers.Input(shape=(1,), name='obf_rate')
# Architecture de categorisation
y = tf.keras.layers.Dense(256, activation='relu', name='cat-dense1')(input_text)
y = tf.keras.layers.Dropout(0.3, name='cat-dropout1')(y)
y = tf.keras.layers.Dense(128, activation='relu', name='cat-dense2')(y)
y = tf.keras.layers.Dropout(0.3, name='cat-dropout2')(y)
y = tf.keras.layers.Dense(64, activation='relu', name='cat-dense3')(y)
y = tf.keras.layers.Dropout(0.3, name='cat-dropout3')(y)

cat = tf.keras.layers.Dense(5, activation='softmax', name='cat')(y)
CategorisationModel = tf.keras.models.Model(inputs=input_text, outputs=cat)
CategorisationModel.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics ='categorical_accuracy') 
# Train the Model

history1 = CategorisationModel.fit(train_features, train_cat, epochs=120, batch_size=32)


# Obtenir la sortie de l'avant-dernière couche du modèle de catégorisation
output_categorisation = CategorisationModel.layers[-2].output
output_categorisation = tf.keras.layers.Dense(64, activation='relu')(output_categorisation)


    # Concaténer les couches d'entrée
concatenated_inputs = tf.keras.layers.concatenate([input_text, input_obf_rate])

    # Définir le reste du modèle
x = tf.keras.layers.Dense(256, activation='relu', name='spam-dense1')(concatenated_inputs)
x = tf.keras.layers.Dropout(0.3, name='spam-dropout1')(x)
x = tf.keras.layers.Dense(128, activation='relu', name='spam-dense2')(x)
x = tf.keras.layers.Dropout(0.3, name='spam-dropout2')(x)
x = tf.keras.layers.Dense(64, activation='relu', name='spam-dense3')(x)
x = tf.keras.layers.Dropout(0.3, name='spam-dropout3')(x)
# Appliquer une couche d'attention
attention = Attention()([x, output_categorisation])


# Ajouter une couche dense pour fusionner les sorties
merged_output = tf.keras.layers.Dense(64, activation='relu', name='merged_dense')(attention)

# Couche de sortie
outputSpam = tf.keras.layers.Dense(1, activation='sigmoid', name='output-spam')(merged_output)

# Créer le modèle final
spam_model = tf.keras.models.Model(inputs=[input_text, input_obf_rate], outputs=outputSpam)

# Compiler et entraîner le modèle
spam_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = spam_model.fit([train_features, train_obf_rate], train_labels, epochs=100, batch_size=32,
                         validation_data=([val_features, val_obf_rate], val_labels))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
# Évaluer le modèle sur les données de test
test_loss, test_accuracy = spam_model.evaluate([test_features, test_obf_rate], test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

    # Prédire les étiquettes pour les données de test
SuperModelpred = spam_model.predict([test_features, test_obf_rate])
SuperModelpred = np.round(SuperModelpred).flatten()

    # Calculer et afficher les métriques
print(f"roc_auc_score : {roc_auc_score(test_labels, SuperModelpred)}")
print(f"f1_score  : {f1_score(test_labels, SuperModelpred)}")
print(f"recall_score : {recall_score(test_labels, SuperModelpred)}")
print(f"precision_score : {precision_score(test_labels, SuperModelpred)}")

    # Afficher la matrice de confusion
cm = confusion_matrix(test_labels, SuperModelpred)
print(cm) 

In [ ]:
df3 = pd.read_csv("/kaggle/input/textspam/text_spam (1).csv")
df3['texteTest'] = df3.iloc[:, 0]
df3 = df3.drop(df3.columns[0], axis=1)

In [ ]:
features = embed.signatures['serving_default'](tf.convert_to_tensor(df3['texteTest']))['outputs']

In [ ]:
df3_labels = spam_model.predict([features, df3["taux_obfuscation"]])
df3_labels = np.round(df3_labels).flatten()

# Ajouter les étiquettes prédites à df3
df3["spam"] = df3_labels

In [ ]:
df3.head()

In [ ]:
spam_count = df3[df3["spam"] == 1].shape[0] 
ham_count = df3[df3["spam"] == 0].shape[0]

print("Nombre d'instances spam :", spam_count) 
print("Nombre d'instances ham :", ham_count)

CODE SUIVANT avec boucle IF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

spam_models = {}

for categorie in ['business', 'politics', 'tech', 'sport', 'entertainment']:
    # Select the data for this category
    df_categorie = df1[df1['category'] == categorie]

    train_texts_categorie = df_categorie['emailclean']
    train_labels_categorie = df_categorie['label']
    train_cat_categorie = df_categorie['category']
    train_category_obf_rate = df_categorie['taux_obfuscation']

    # Split the data into train, validation, and test sets
    train_texts, test_texts, train_labels, test_labels, train_cat, test_cat, train_obf_rate, test_obf_rate = train_test_split(
        train_texts_categorie, train_labels_categorie, train_cat_categorie, train_category_obf_rate, 
        test_size=0.3, stratify=train_labels_categorie, random_state=42
    )

    train_texts, val_texts, train_labels, val_labels, train_obf_rate, val_obf_rate = train_test_split(
        train_texts, train_labels, train_obf_rate,
        test_size=0.3, stratify=train_labels, random_state=42
    )

    # Create an instance of TfidfVectorizer and fit it on the training data
    vectorizer = TfidfVectorizer()
    train_features = vectorizer.fit_transform(train_texts)
    test_features = vectorizer.transform(test_texts)

    # Define inputs for the model
    text_input = tf.keras.layers.Input(shape=(train_features.shape[1],), dtype=tf.float32, name='text')
    obf_rate_input = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='obf_rate')

    # Concatenate the features
    concatenated = tf.keras.layers.Concatenate()([text_input, obf_rate_input])

    dense = Dense(256, activation='relu', name='dense')(concatenated)
    dropout = Dropout(0.1, name='dropout')(dense)
    output = Dense(1, activation='sigmoid', name='output')(dropout)

    spam_model = tf.keras.Model(inputs=[text_input, obf_rate_input], outputs=[output])

    # Compile and train the model
    spam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Convert the features to tensors
    train_features_tensor = tf.convert_to_tensor(train_features.toarray(), dtype=tf.float32)
    val_features_tensor = tf.convert_to_tensor(vectorizer.transform(val_texts).toarray(), dtype=tf.float32)
    test_features_tensor = tf.convert_to_tensor(test_features.toarray(), dtype=tf.float32)

    # Use the cache method to cache the data in memory to speed up the model training
    train_data = tf.data.Dataset.from_tensor_slices(((train_features_tensor, train_obf_rate), train_labels)).cache().batch(32)
    val_data = tf.data.Dataset.from_tensor_slices(((val_features_tensor, val_obf_rate), val_labels)).cache().batch(32)
    test_data = tf.data.Dataset.from_tensor_slices(((test_features_tensor, test_obf_rate), test_labels)).batch(32)

    # Define the EarlyStopping callback
    history = spam_model.fit(train_data, epochs=40, batch_size=32, validation_data=val_data)
    
    
    test_loss, test_accuracy = spam_model.evaluate(test_data)

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    y_pred = spam_model.predict(test_data)
    y_pred = np.round(y_pred).flatten()
    
    
   
    # Calculate and print the metrics
    print(f"roc_auc_score for {categorie}: {roc_auc_score(test_labels, y_pred)}")
    print(f"f1_score for {categorie}: {f1_score(test_labels, y_pred)}")
    print(f"recall_score for {categorie}: {recall_score(test_labels, y_pred)}")
    print(f"precision_score for {categorie}: {precision_score(test_labels, y_pred)}")
    # Print the confusion matrix
    cm = confusion_matrix(test_labels, y_pred)
    print(f"Confusion Matrix for {categorie}:")
    print(cm)
    

    spam_models[categorie] = spam_model

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, MultiHeadAttention 
from tensorflow.keras.layers import Concatenate
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

spam_models = {}
categories = ['business', 'politics', 'tech', 'sport', 'entertainment']

for category in categories:
    # Sélectionner les données pour cette catégorie
    df_category = df1[df1['category'] == category]

    train_texts_category = df_category['emailclean']
    train_labels_category = df_category['label']
    train_cat_category = df_category['category']
    train_category_obf_rate = df_category['taux_obfuscation']

    # Diviser les données en ensembles d'entraînement, de validation et de test
    train_texts, test_texts, train_labels, test_labels, train_cat, test_cat, train_obf_rate, test_obf_rate = train_test_split(
        train_texts_category, train_labels_category, train_cat_category, train_category_obf_rate,
        test_size=0.3, stratify=train_labels_category, random_state=42
    )

    train_texts, val_texts, train_labels, val_labels, train_obf_rate, val_obf_rate = train_test_split(
        train_texts, train_labels, train_obf_rate,
        test_size=0.3, stratify=train_labels, random_state=42
    )

    # Préparer les données d'entraînement, de validation et de test avec l'embedding Universal Sentence Encoder
    train_features = embed.signatures['serving_default'](tf.convert_to_tensor(train_texts))['outputs']
    val_features = embed.signatures['serving_default'](tf.convert_to_tensor(val_texts))['outputs']
    test_features = embed.signatures['serving_default'](tf.convert_to_tensor(test_texts))['outputs']

    # Définir les couches d'entrée pour les textes et le taux d'obfuscation
    input_text = tf.keras.layers.Input(shape=(train_features.shape[1],), name='text')
    input_obf_rate = tf.keras.layers.Input(shape=(1,), name='obf_rate')

    # Concaténer les couches d'entrée
    concatenated_inputs = tf.keras.layers.concatenate([input_text, input_obf_rate])


    # Définir le reste du modèle
    x = tf.keras.layers.Dense(256, activation='relu', name='dense1')(concatenated_inputs)
    x = tf.keras.layers.Dropout(0.3, name='dropout1')(x)
    x = tf.keras.layers.Dense(128, activation='relu', name='dense2')(x)
    x = tf.keras.layers.Dropout(0.3, name='dropout2')(x)
    x = tf.keras.layers.Dense(64, activation='relu', name='dense3')(x)
    x = tf.keras.layers.Dropout(0.3, name='dropout3')(x)

    # Ajouter une couche d'expansion pour ajuster la dimension du tenseur
    x = tf.keras.layers.Reshape((1, -1))(x)
    # Ajouter une couche Multi-Head Attention
    attention = MultiHeadAttention(num_heads=8, key_dim=120)
    x = attention(x, x)

    # Réduire les dimensions du tenseur pour la couche BatchNormalization
    x = tf.reduce_mean(x, axis=1)


    x = tf.keras.layers.BatchNormalization()(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

    spam_model = tf.keras.models.Model(inputs=[input_text, input_obf_rate], outputs=output)
    # Compiler et entraîner le modèle
    spam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy', metrics=['accuracy'])
    history = spam_model.fit(
        [train_features, train_obf_rate], train_labels,
        epochs=30, batch_size=32,
        validation_data=([val_features, val_obf_rate], val_labels)
    )

    # Évaluer le modèle sur les données de test
    test_loss, test_accuracy = spam_model.evaluate([test_features, test_obf_rate], test_labels)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

    # Prédire les étiquettes pour les données de test
    y_pred = spam_model.predict([test_features, test_obf_rate])
    y_pred = np.round(y_pred).flatten()

    # Calculer et afficher les métriques
    print(f"roc_auc_score for {category}: {roc_auc_score(test_labels, y_pred)}")
    print(f"f1_score for {category}: {f1_score(test_labels, y_pred)}")
    print(f"recall_score for {category}: {recall_score(test_labels, y_pred)}")
    print(f"precision_score for {category}: {precision_score(test_labels, y_pred)}")

    # Afficher la matrice de confusion
    cm = confusion_matrix(test_labels, y_pred)
    print(f"Confusion Matrix for {category}:")
    print(cm)

    spam_models[category] = spam_model

In [ ]:
for category, model in spam_models.items():
    print(f"Architecture du modèle pour la catégorie '{category}':")
    print(model.summary())
    print('\n')

In [ ]:
from tensorflow.keras.utils import plot_model

# Supposons que votre modèle s'appelle "spam_model"
plot_model(spam_model, to_file='architecture.png', show_shapes=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import tensorflow as tf

spam_models = {}

for categorie in ['business', 'politics', 'tech', 'sport', 'entertainment']:
    # Select the data for this category
    df_categorie = df1[df1['category'] == categorie]

    train_texts_categorie = df_categorie['text']
    train_labels_categorie = df_categorie['label']
    train_cat_categorie = df_categorie['category']
    train_category_obf_rate = df_categorie['taux_obfuscation']

    # Split the data into train, validation, and test sets
    train_texts, test_texts, train_labels, test_labels, train_cat, test_cat, train_obf_rate, test_obf_rate = train_test_split(
        train_texts_categorie, train_labels_categorie, train_cat_categorie, train_category_obf_rate, 
        test_size=0.3, stratify=train_labels_categorie, random_state=42
    )

    train_texts, val_texts, train_labels, val_labels, train_obf_rate, val_obf_rate = train_test_split(
        train_texts, train_labels, train_obf_rate,
        test_size=0.3, stratify=train_labels, random_state=42
    )

    # Create an instance of CountVectorizer and fit it on the training data
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(train_texts)

    # Conversion of texts to vector representations using CountVectorizer
    train_features_count = count_vectorizer.transform(train_texts).toarray()
    val_features_count = count_vectorizer.transform(val_texts).toarray()
    test_features_count = count_vectorizer.transform(test_texts).toarray()

    # Concatenation of CountVectorizer vector representations with other features
    train_features_concat = np.concatenate([train_features_count, train_obf_rate[:, np.newaxis]], axis=1)
    val_features_concat = np.concatenate([val_features_count, val_obf_rate[:, np.newaxis]], axis=1)
    test_features_concat = np.concatenate([test_features_count, test_obf_rate[:, np.newaxis]], axis=1)

    # Define the model architecture
    spam_model = tf.keras.Sequential()
    spam_model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(train_features_concat.shape[1],)))
    spam_model.add(tf.keras.layers.Dropout(0.1))
    spam_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    spam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = spam_model.fit(train_features_concat, train_labels, epochs=40, batch_size=32, validation_data=(val_features_concat, val_labels))
 
    # Evaluation du modèle avec les données de test
    test_loss, test_accuracy = spam_model.evaluate(test_features_concat, test_labels)

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

    # Prédiction des classes avec le modèle
    y_pred = spam_model.predict(test_features_concat)
    y_pred = np.round(y_pred).flatten()
    # Calculate and print the metrics
    print(f"roc_auc_score for {categorie}: {roc_auc_score(test_labels, y_pred)}")
    print(f"f1_score for {categorie}: {f1_score(test_labels, y_pred)}")
    print(f"recall_score for {categorie}: {recall_score(test_labels, y_pred)}")
    print(f"precision_score for {categorie}: {precision_score(test_labels, y_pred)}")

    # Print the confusion matrix
    cm = confusion_matrix(test_labels, y_pred)
    print(f"Confusion Matrix for {categorie}:")
    print(cm)

    # Enregistrer le modèle pour cette catégorie
    spam_models[categorie] = spam_model

In [ ]:
import pydot
from tensorflow.keras.utils import model_to_dot
# Générer une représentation graphique du modèle
dot = model_to_dot(spam_model)
png_bytes = dot.create_png(prog='dot')

with open(f"{category}_model.png", "wb") as f:
    f.write(png_bytes)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
# Afficher la représentation graphique du modèle
plt.figure(figsize=(10, 10))
img = plt.imread(f"{category}_model.png")
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:

import matplotlib.pyplot as plt

# Créer un répertoire pour enregistrer les modèles
models_directory = '/kaggle/working/'
import tensorflow as tf
os.makedirs(models_directory, exist_ok=True)

# Entraîner et enregistrer les modèles
for category, model in spam_models.items():
    history = model.history
    epochs = range(1, len(history['accuracy']) + 1)

    # Enregistrer le modèle
    model.save(f'{models_directory}/{category}.h5')

    # Tracer les courbes de précision et de perte
    plt.figure()
    plt.plot(epochs, history.history['accuracy'], 'r', label='Training Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], 'b', label='Validation Accuracy')
    plt.title(f'Accuracy - {category}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'{models_directory}/{category}_accuracy.png')
    plt.show()

    plt.figure()
    plt.plot(epochs, history['loss'], 'r', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'b', label='Validation Loss')
    plt.title(f'Loss - {categorie}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{models_directory}/{category}_loss.png')
    plt.show()


In [ ]:
category_counts = test_cat.value_counts()
print(category_counts)

In [ ]:

spam = []

for i in range(len(Xnew)):
    if Znew[i] == 'business':
        ar = np.array([Xnew[i]])
        df10 = pd.DataFrame(ar, columns=['text'])
        new_text_vector = vectorizer.transform(df10['text']).toarray()
        y_predicted_bus = loaded_model.predict(new_text_vector)
        y_predicted_bus = y_predicted_bus.flatten()
        spam.append(np.where(y_predicted_bus > 0.5, 1, 0))
    elif Znew[i] == 'tech':
        ar = np.array([Xnew[i]])
        df10 = pd.DataFrame(ar, columns=['text'])
        new_text_vector = vectorizer.transform(df10['text']).toarray()
        y_predicted_tech = loaded_model.predict(new_text_vector)
        y_predicted_tech = y_predicted_tech.flatten()
        spam.append(np.where(y_predicted_tech > 0.5, 1, 0))
    elif Znew[i] == 'entertainment':
        ar = np.array([Xnew[i]])
        df10 = pd.DataFrame(ar, columns=['text'])
        new_text_vector = vectorizer.transform(df10['text']).toarray()
        y_predicted_Entert = loaded_model.predict(new_text_vector)
        y_predicted_Entert = y_predicted_Entert.flatten()
        spam.append(np.where(y_predicted_Entert > 0.5, 1, 0))
    elif Znew[i] == 'sport':
        ar = np.array([Xnew[i]])
        df10 = pd.DataFrame(ar, columns=['text'])
        new_text_vector = vectorizer.transform(df10['text']).toarray()
        y_predicted_spo = loaded_model.predict(new_text_vector)
        y_predicted_spo = y_predicted_spo.flatten()
        spam.append(np.where(y_predicted_spo > 0.5, 1, 0))
    else:
        ar = np.array([Xnew[i]])
        df10 = pd.DataFrame(ar, columns=['text'])
        new_text_vector = vectorizer.transform(df10['text']).toarray()
        y_predicted_polit = loaded_model.predict(new_text_vector)
        y_predicted_polit = y_predicted_polit.flatten()
        spam.append(np.where(y_predicted_polit > 0.5, 1, 0))

In [ ]:
dfnew['spam']=pd.DataFrame(spam)

In [ ]:
New_df_emails = dfnew.to_csv('/kaggle/working/BBC-TEXT-NEW.csv', index = True)

In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)